In [1]:
import geopandas as gpd
import os
import zipfile as zf
import geopandas
from shapely.geometry import shape
import gzip 
import urllib.request
import xarray as xr
import io
import shapefile
import pandas as pd
import json

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [90]:
df = pd.read_pickle("GDP_temperature_anomalies_1947_USA.pkl")

In [91]:
df

,time,timeMin,timeMax,gdp
0,1947-01-15,-6.66,4.33,2033.061000
1,1947-02-15,-3.48,3.51,2031.253667
2,1947-03-15,-3.64,4.59,2029.446333
3,1947-04-15,-1.90,1.61,2027.639000
4,1947-05-15,-2.55,2.46,2026.243333
...,...,...,...,...
887,2020-12-15,-0.27,5.58,18989.854000
888,2021-01-15,0.48,7.10,19087.568000
889,2021-02-15,-4.14,1.27,NaN
890,2021-03-15,-1.28,5.49,NaN


In [92]:
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df = df.set_index(df['time'])
df = df.sort_index()
df =  df.drop(["time","index","gdp"], axis =1)

In [93]:
df

,timeMin,timeMax
time,,
1947-01-15,-6.66,4.33
1947-02-15,-3.48,3.51
1947-03-15,-3.64,4.59
1947-04-15,-1.90,1.61
1947-05-15,-2.55,2.46
...,...,...
2020-12-15,-0.27,5.58
2021-01-15,0.48,7.10
2021-02-15,-4.14,1.27


In [97]:
892*0.75

669.0

In [98]:
train,test = df[1:669], df[669:] 

In [99]:
train

,timeMin,timeMax
time,,
1947-02-15,-3.48,3.51
1947-03-15,-3.64,4.59
1947-04-15,-1.90,1.61
1947-05-15,-2.55,2.46
1947-06-15,-1.95,1.63
...,...,...
2002-05-15,-2.23,2.51
2002-06-15,-0.40,2.27
2002-07-15,-0.47,3.18


Scaling dataset

In [103]:
pd.options.mode.chained_assignment = None  # default='warn'
min_max_scaler = MinMaxScaler()
train[["timeMin", "timeMax"]] = min_max_scaler.fit_transform(train[["timeMin", "timeMax"]])

In [105]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [106]:
n_past = 50
n_future = 1
n_features = 2

In [107]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [108]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_12 (LSTM)                  [(None, 100), (None, 41200       input_5[0][0]                    
__________________________________________________________________________________________________
repeat_vector_4 (RepeatVector)  (None, 1, 100)       0           lstm_12[0][0]                    
__________________________________________________________________________________________________
lstm_13 (LSTM)                  (None, 1, 100)       80400       repeat_vector_4[0][0]            
                                                                 lstm_12[0][1]         

In [109]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_14 (LSTM)                  [(None, 50, 100), (N 41200       input_6[0][0]                    
__________________________________________________________________________________________________
lstm_15 (LSTM)                  [(None, 100), (None, 80400       lstm_14[0][0]                    
__________________________________________________________________________________________________
repeat_vector_5 (RepeatVector)  (None, 1, 100)       0           lstm_15[0][0]                    
______________________________________________________________________________________

In [110]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])

Epoch 1/25
20/20 [==============================] - 1s 69ms/step - loss: 0.0316 - val_loss: 2.2823
Epoch 2/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0106 - val_loss: 2.3079
Epoch 3/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0103 - val_loss: 2.2752
Epoch 4/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0102 - val_loss: 2.2601
Epoch 5/25
20/20 [==============================] - 1s 27ms/step - loss: 0.0101 - val_loss: 2.2517
Epoch 6/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0100 - val_loss: 2.2302
Epoch 7/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0099 - val_loss: 2.2328
Epoch 8/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0098 - val_loss: 2.2379
Epoch 9/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0097 - val_loss: 2.2381
Epoch 10/25
20/20 [==============================] - 1s 28ms/step - loss: 0.0098 - val_loss: 2.2464
Epoch 11/

In [81]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

In [82]:
pred_e1d1

array([[[0.70154095, 0.2743775 ]],

       [[0.6968091 , 0.26876283]],

       [[0.7036882 , 0.2569926 ]],

       [[0.7103477 , 0.25682935]],

       [[0.7176047 , 0.25358194]],

       [[0.71442527, 0.24424745]],

       [[0.70784885, 0.23522367]],

       [[0.7085497 , 0.23900618]],

       [[0.71580034, 0.2448578 ]],

       [[0.7158285 , 0.24768598]],

       [[0.73756677, 0.26851898]],

       [[0.77143973, 0.29649365]],

       [[0.8001413 , 0.3055137 ]],

       [[0.8249328 , 0.30238512]],

       [[0.826325  , 0.303078  ]],

       [[0.8166557 , 0.2917668 ]],

       [[0.81361365, 0.28168508]],

       [[0.80969054, 0.27880365]],

       [[0.79931015, 0.2716334 ]],

       [[0.78804034, 0.26449043]],

       [[0.7879341 , 0.27014244]],

       [[0.80124503, 0.27882406]],

       [[0.7811891 , 0.2788387 ]],

       [[0.77106255, 0.27620932]],

       [[0.77718467, 0.281689  ]],

       [[0.7530216 , 0.27791268]],

       [[0.756707  , 0.2723412 ]],

       [[0.7452439 , 0.25662

In [87]:
pred_e2d2

array([[[0.683442  , 0.24414946]],

       [[0.6857188 , 0.2390319 ]],

       [[0.69353384, 0.23129682]],

       [[0.6987966 , 0.22925086]],

       [[0.7043388 , 0.22727685]],

       [[0.7043925 , 0.2252102 ]],

       [[0.70122385, 0.22445054]],

       [[0.69939286, 0.22815879]],

       [[0.70118594, 0.23217924]],

       [[0.7003465 , 0.23550804]],

       [[0.7097796 , 0.24281998]],

       [[0.72790134, 0.25099128]],

       [[0.74858654, 0.25233212]],

       [[0.7694125 , 0.2502076 ]],

       [[0.77723503, 0.25181845]],

       [[0.77901256, 0.24963488]],

       [[0.7815234 , 0.24756809]],

       [[0.7809533 , 0.24874698]],

       [[0.7762971 , 0.24905778]],

       [[0.7697781 , 0.24941234]],

       [[0.76686776, 0.25326055]],

       [[0.77140915, 0.25699806]],

       [[0.760885  , 0.2595604 ]],

       [[0.755247  , 0.25859097]],

       [[0.75713015, 0.25866267]],

       [[0.7449464 , 0.2571182 ]],

       [[0.7465641 , 0.25186214]],

       [[0.74089456, 0.24545

Trying to rescale it back:

In [89]:
pred1_e2d2 = min_max_scaler.inverse_transform(pred_e1d1)

ValueError: Found array with dim 3. Estimator expected <= 2.